In [ ]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

In [ ]:
cap = cv2.VideoCapture(0)
detector = FaceMeshDetector(maxFaces = 1)


plotY = LivePlot(640,360,[25,50],invert = True)
idList = [22, 23, 24 ,26, 110 ,157, 158 , 159 ,160 ,161, 130, 243]

ratioList = []
BlinkCounter = 0
counter = 0
color = (255, 0, 255 )

while True:
    success,img = cap.read()
    img, faces = detector.findFaceMesh(img, draw = False)

    if faces:
        face = faces[0]
        pointLeft = face[145]
        pointRight = face[374]
        leftup = face[159]
        leftdown  = face[23]
        leftLeft = face[130]
        leftright = face[243]
        lenVer,_ = detector.findDistance(leftup, leftdown)
        lenhor,_ = detector.findDistance(leftLeft, leftright)
        
        # for id in idList:
        #     cv2.circle(img, face[id], 3 ,color, cv2.FILLED)
        # cv2.line(img , leftup,leftdown, (0,200,0),3)
        # cv2.line(img , leftLeft,leftright, (0,200,0),3)

        ratio = (lenVer/lenhor)*100
        ratioList.append((ratio))
        if len(ratioList)>5:
            ratioList.pop(0)
        ratioAvg = sum(ratioList) / len(ratioList)

        if ratioAvg < 31 and counter == 0:
            BlinkCounter +=1 
            color = (0,200,0)
            counter = 1
        if counter != 0 :
            counter +=1 
            if counter > 15:
                counter = 0
                color = (255,0,255)

        cvzone.putTextRect(img, f"BlinkCount:{BlinkCounter}", (10,50),colorR=color)
        imgplot = plotY.update(ratioAvg,color)        
    
        # cv2.line(img, pointLeft,pointRight,(255,0,255), 3 )
        # cv2.circle(img, pointLeft,5,(255,0,255),cv2.FILLED)
        # cv2.circle(img, pointRight,5,(255,0,255),cv2.FILLED)
        
        w,_ = detector.findDistance(pointLeft , pointRight)
        # print(w)
        W = 6.3 #average value for male,Female
        # d = 50
        # f = (w*d)/W
        # print(f)   

        f = 700
        d = (W*f)/w
        # print(d)

        cvzone.putTextRect(img , f"Dept: {int(d)}cm ",(face[10][0]-100, face[10][1]- 50 ),scale = 2)
        img = cv2.resize(img, (640, 360))
        imgstack = cvzone.stackImages([img,imgplot], 2, 1)
    else:
        img = cv2.resize(img ,(640 ,360))
        imgstack = cvzone.stackImages([img,img], 2, 1)


    cv2.imshow("img", imgstack)
    cv2.waitKey(1)

